In [1]:
from probemapper.registration import ANTSPointTransformation, ANTSTransformationStages
from probemapper.io import save_nifti
import os, subprocess
import numpy as np
import pandas as pd

In [2]:
rootdir = "/bucket/ReiterU/DBS/data/20220529_P191_laqueus_DiI_LM13/"
resultdir = os.path.join(rootdir, "analysis")
if not os.path.exists(resultdir):
    subprocess.run(["ssh", "deigo", "mkdir", resultdir])

In [3]:
cwd = os.path.basename(os.getcwd())
tmpdir = os.path.join("/flash/ReiterU/tmp/", cwd, f"indexing")
if not os.path.exists(tmpdir):
    os.makedirs(tmpdir)

### Transform channel coordinates

In [4]:
T = ANTSPointTransformation(
    ants_path="/apps/unit/ReiterU/ANTs/2.3.5/bin/",
    num_threads=8
)
T.set_transformations([
    ANTSTransformationStages(transformation=os.path.join(rootdir, "analysis/ants/F2M_0GenericAffine.mat"), inverse=True),
    ANTSTransformationStages(transformation=os.path.join(rootdir, "analysis/ants/F2M_1InverseWarp.nii.gz"), inverse=False),
])
T.set_moving_points(os.path.join(rootdir, "analysis/skeleton/channel_skeleton.csv"))
T.set_outname(os.path.join(tmpdir, "channel_LUT.csv"))

In [5]:
T.run()

['/apps/unit/ReiterU/ANTs/2.3.5/bin/antsApplyTransformsToPoints', '-d', '3', '-i', '/bucket/ReiterU/DBS/data/20220529_P191_laqueus_DiI_LM13/analysis/skeleton/channel_skeleton.csv', '-o', '/flash/ReiterU/tmp/P191/indexing/channel_LUT.csv', '-t', '[/bucket/ReiterU/DBS/data/20220529_P191_laqueus_DiI_LM13/analysis/ants/F2M_0GenericAffine.mat,1]', '-t', '/bucket/ReiterU/DBS/data/20220529_P191_laqueus_DiI_LM13/analysis/ants/F2M_1InverseWarp.nii.gz']



### Query region ID

In [6]:
from probemapper import skeletonization

In [7]:
df = pd.read_csv(os.path.join(tmpdir, "channel_LUT.csv"))
coords = np.array([df["Z"], df["Y"], df["X"]])

In [9]:
regions = skeletonization.query_region_id2(coords,
                     "/bucket/ReiterU/DBS/atlas/O_Laqueus_v0.1/Slicer_3D/Segmentation.seg.nrrd",
                     100, coords.shape[1]-100, 10)

In [10]:
df["Region"] = regions

In [11]:
df.insert(0, "channel", df.pop("channel"))

In [12]:
display(df)

,channel,X,Y,Z,X_org,Y_org,Z_org,Region
0,-100,2366.92,2665.38,1845.15,1641.95,2811.640,1555.96,8
1,-99,2368.43,2666.71,1855.71,1642.74,2807.610,1564.74,8
2,-98,2370.14,2667.84,1866.44,1643.53,2803.580,1573.52,8
3,-97,2371.99,2668.79,1877.25,1644.32,2799.550,1582.31,8
4,-96,2373.91,2669.59,1888.01,1645.12,2795.520,1591.09,8
...,...,...,...,...,...,...,...,...
346,246,2806.23,2040.13,4663.94,1831.29,605.292,4017.77,3
347,247,2806.94,2033.59,4669.76,1831.58,596.414,4021.97,3
348,248,2807.82,2027.00,4675.78,1831.98,587.513,4026.24,3
349,249,2808.91,2020.34,4682.06,1832.48,578.586,4030.57,3


In [13]:
df.to_csv(os.path.join(tmpdir, "channel_LUT.csv"), index=False)

In [15]:
sk = skeletonization.generate_skeleton_image(coords[:, 100:],
                             "/bucket/ReiterU/DBS/atlas/O_Laqueus_v0.1/Slicer_3D/Segmentation.seg.nrrd",
                             10)

In [16]:
sk = sk.astype("uint8")
save_nifti(sk, os.path.join(tmpdir, "probe_image.nii.gz"), 1, 1, 1)

In [17]:
subprocess.run(["scp", "-r", tmpdir, f"deigo:{resultdir}"])

CompletedProcess(args=['scp', '-r', '/flash/ReiterU/tmp/P191/indexing', 'deigo:/bucket/ReiterU/DBS/data/20220529_P191_laqueus_DiI_LM13/analysis'], returncode=0)